In [468]:
import os
import torch
import operator
import matplotlib as plt
from pylab import *
import torch.nn as nn
import torchaudio
import numpy as np
import pandas as pd
from tqdm import notebook

from glob import glob

In [3]:
data_dir = "D:\\Music\\Sample Packs\\"

In [4]:
paths = [path.lower() for path in glob(f'{data_dir}/**/*.wav', recursive=True)]
len(paths)
print(paths[123])

d:\music\sample packs\1. delifb chill future bass & trap sounds\kicks\kick 9.wav


### 24 Categories, 17 Genres, 46 Styles

In [398]:
count = 0
category = ["drum", "kick", "snare", "foley", "percussion", "clap", "cymbal", "crash", "tom", "rim", "bell", "tambourine",
            "hat", "closed_hat", "open_hat", "ride", "depot", "snap", "shake", "reverse_crash", "impact"]
genres = ["future_bass", "dubstep", "hitech", "trap", "hardcore", "riddim", "uk_garage","acoustic", "bass_house", "pop",
          "house", "future_bounce", "future_house", "tropical_house", "edm", "rnb", "reggae", "hip_hop", "hardstyle", "orchestral", "cinematic"]
style = ["kshmr", "future", "acoustic", "spicy", "tropical", "high", "low", "electro", "808", "melodic", "weird", "punch", "kawaii",
         "crunch", "epic", "wet", "lofi", "deep", "fat", "tight", "vintage", "top", "gated", "closet", "pan", "heavy", "light" , "scratch", "roll", "noise",
         "hard", "soft", "stomp", "short", "thick", "anime", "bright", "glitch", "beatbox", "dirty", "distort", "droplet", "reverse", "rattle", 
         "djembe", "chime", "stadium"]

delete = ["loop", "vox", "guitar", "buildup", "fill", "construction", "riser", "impact", "brass"]

for path in paths :
    if any(keyword in path for keyword in category) and not any(keyword in path for keyword in delete) :
        count+=1
print(count)

9379


In [247]:
keywords = {}
df = pd.DataFrame(columns=["data_dir", "category", "genre", "style"])
df

,data_dir,category,genre,style


In [248]:
pbar = notebook.tqdm(paths, desc='keyword processing...')
for path in pbar : 
    if not any(keyword in path for keyword in category) or any(keyword in path for keyword in delete) : continue
    df_row = {}
    df_row["data_dir"] = path
    string = path
    
    if "_" in string : string = string.replace("_", " ") 
    if "reverse crash" in string : string = string.replace("reverse crash", "reverse_crash")
    
    chs = ["closed hat", "close hat", "closed hihat", "close hihat", "hat closed", "hihat closed",
           "closed hats", "close hats", "closed hihats", "close hihats", "hats closed", "hihats closed",
          "closed hh", "clhh", "ukg ch", "2sg ch"]
    ohs = ["opened hat", "open hat", "opened hihat", "open hihat", "hat opened", "hihat opened",
           "opened hats", "open hats", "opened hihats", "open hihats", "hats opened", "hihats opened",
           "open hh", "ophh", "ukg oh", "2sg oh"]
    
    for ch in chs :
        string = string.replace(ch, "closed_hat")
    for oh in ohs :
        string = string.replace(oh, "open_hat")
    
    if "snr" in string : string = string.replace("snr", "snare")
    if "percs" in string : string = string.replace("percs", "percussion")
    if "crsh" in string : string = string.replace("crsh", "crash")
    if "prc" in string : string = string.replace("prc", "percussion")
    if "clp" in string : string = string.replace("clp", "clap")
    if "kawaii bass" in string : string = string.replace("kawaii bass", "future bass")
    if "clap & snares" in string : string = string.replace("clap & snares", "")
    if "snares and claps" in string : string = string.replace("snares and claps", "")
    if "pop_" in string : string = string.replace("pop_", "pop ")
        
    specialChars = "\.-,+*%&@^"
    for char in specialChars :
        string = string.replace(char, ' ')
    
    types = []
    for type_ in category :
        if type_ in string :
            types.append(type_)
    type_ = ",".join(types)
    df_row["category"] = type_
    
    
    gens = []
    if "benzie" in string :
        gens.append("trap"); gens.append("hip_hop")
    if "hbd" in string :
        gens.append("dubstep")
        
    for genre in genres :
        genre_ = genre.replace(" ", "_")
        if genre_ == "pop_" : genre_ = "pop"
        if genre in string :
            string = string.replace(genre, genre_)
            gens.append(genre_)
    if "aethral" in string :
        if not "hardcore" in gens :
            gens.append("hardcore")
    genre_ = ",".join(gens)
    df_row["genre"] = genre_
    
    
    sty = []
    for style_ in style :
        if style_ in string :
            sty.append(style_)
    style_ = ",".join(sty)
    df_row["style"] = style_
    
    
    df = df.append(df_row, ignore_index=True)
    df.replace(np.nan, '', regex=True)

        
    strlist = sorted(list(filter(None, string.split(' ')[4:-1])))
    
    for i in range(len(strlist)) :
        if i>=1 and strlist[i-1] == strlist[i] :
            continue
        word = strlist[i]
        keywords[word] = keywords.get(word, 0) + 1

keyword processing...:   0%|          | 0/30769 [00:00<?, ?it/s]

In [ ]:
keywords = sorted(keywords.items(), key=operator.itemgetter(1), reverse=True)
print(len(keywords))
#print(keywords)

for key in keywords :
    if key[1] <= 5 : pass
    print(key[0], " : " , key[1])

In [249]:
df.to_csv("metadata.csv")
df

,data_dir,category,genre,style
0,d:\music\sample packs\1. delifb chill future b...,snare,"future_bass,trap",future
1,d:\music\sample packs\1. delifb chill future b...,tom,"future_bass,trap",future
2,d:\music\sample packs\1. delifb chill future b...,drum,"future_bass,trap",future
3,d:\music\sample packs\1. delifb chill future b...,drum,"future_bass,trap",future
4,d:\music\sample packs\1. delifb chill future b...,drum,"future_bass,trap",future
...,...,...,...,...
9374,d:\music\sample packs\w.a.prod mega pack\wapro...,snare,"bass_house,house",
9375,d:\music\sample packs\w.a.prod mega pack\wapro...,snare,"bass_house,house",
9376,d:\music\sample packs\w.a.prod mega pack\wapro...,snare,"bass_house,house",
9377,d:\music\sample packs\w.a.prod mega pack\wapro...,snare,"bass_house,house",


In [60]:
def embedding(num, dim, max_norm=1) :
    return nn.Embedding(num_embeddings=num, embedding_dim=dim, max_norm=max_norm)

def dense_layer(input_, output_) :
    return nn.Linear(input_, output_)


In [399]:
vocab_category = {cat: i+1 for i, cat in enumerate(category)}
vocab_category[None] = 0
vocab_category
vocab_genre = {cat: i+1 for i, cat in enumerate(genres)}
vocab_genre[None] = 0
vocab_style    = {sty: i+1 for i, sty in enumerate(style)}
vocab_style[None] = 0

vocab = vocab_category, vocab_genre, vocab_style
vocab

({'drum': 1,
  'kick': 2,
  'snare': 3,
  'foley': 4,
  'percussion': 5,
  'clap': 6,
  'cymbal': 7,
  'crash': 8,
  'tom': 9,
  'rim': 10,
  'bell': 11,
  'tambourine': 12,
  'hat': 13,
  'closed_hat': 14,
  'open_hat': 15,
  'ride': 16,
  'depot': 17,
  'snap': 18,
  'shake': 19,
  'reverse_crash': 20,
  'impact': 21,
  None: 0},
 {'future_bass': 1,
  'dubstep': 2,
  'hitech': 3,
  'trap': 4,
  'hardcore': 5,
  'riddim': 6,
  'uk_garage': 7,
  'acoustic': 8,
  'bass_house': 9,
  'pop': 10,
  'house': 11,
  'future_bounce': 12,
  'future_house': 13,
  'tropical_house': 14,
  'edm': 15,
  'rnb': 16,
  'reggae': 17,
  'hip_hop': 18,
  'hardstyle': 19,
  'orchestral': 20,
  'cinematic': 21,
  None: 0},
 {'kshmr': 1,
  'future': 2,
  'acoustic': 3,
  'spicy': 4,
  'tropical': 5,
  'high': 6,
  'low': 7,
  'electro': 8,
  '808': 9,
  'melodic': 10,
  'weird': 11,
  'punch': 12,
  'kawaii': 13,
  'crunch': 14,
  'epic': 15,
  'wet': 16,
  'lofi': 17,
  'deep': 18,
  'fat': 19,
  'tight': 20